# Does increasing the tax base lower property taxes?

## Background

A large commercial development might be coming to my town. It's a development that many residents oppose. One argument in favor of the development is that it would increase the tax base, and with more properties to tax, the town could lessen the burden on homeowners by lowering property taxes.

This seems like a persuasive argument, but I wonder if there is any data to back it up.

## Research Questions

1. Does my town have a history of lowering the mill rate when the grand list increases?
3. How has Connecticut's average grand list and mill rate changed over time?

## Findings

* Since 2014, Connecticut's average grand list and mill rate have frequently increased together. The ideal scenario, where a grand list increases and the mill rate decreases, has occured less often. 
* For several years, as my town's grand list increased, so did its mill rate. However, in the last few years, the grand list has continued to increase while the mill rate has started to fall.
* Similarly, the average grand list and mill rate in Connecticut increased together for several years, and only the mill rate has recently started to fall.

## Glossary

* Mill Rate = Grand Levy / Grand List
* Grand Levy: The amount of revenue required to operate the City for the fiscal year. (1)
* Grand List: The total assessed value of all taxable property.
* Grand List Year = The term used to label the taxes for a given fiscal year. The grand list is as of October 1; the assessor determines what that is by February 1. In May, the tax rates are set, and in July and the following January, tax bills come due for that grand list year.
* Fiscal Year: July 1 through June 30. Taxes are based on the Grand List of the preceding October 1.

1. Grand Levy = Mill Rate x Grand List

## Notes

* Some towns have service districts with different mill rates. Service districts were not included in this analysis.
* The fiscal_year is the ending year. For example, fiscal_year 2018 is fiscal year 2017-2018.
* Mill rates throughout a town can vary depending on services received and district location.

## Preprocessing

* Imported packages.
* Used an API to import mill rate and grand list data from the State of Connecticut's website.
* Created new keys and used an inner join to combine the data.
* Inspected and dropped observations based on null and outlier values. (1) 
* Copied mill_rate_real_personal and mill_rate values into a new column. (2)
* Converted data types to numeric as needed.
* Sorted rows by town and grand list year in order to calculate percent changes.
* Calculated percent changes to mill rate and grand list and saved to new columns. (3)

1. Each observation must have a mill_rate_real_personal value or a mill_rate value. Stamford's 2019 data was dropped because some values were too small to be correct.
2. In fiscal year 2021, some towns began using the mill_rate_real_personal label instead of mill_rate.
3. I'm using total_equalized values for the grand lists. The total_equalized values exclude non-taxable property.

In [ ]:
# import packages
import pandas as pd
import numpy as np
from scipy import stats
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
from sodapy import Socrata

In [ ]:
# import mill rates
client_mill = Socrata('data.ct.gov', None)
results_mill = client_mill.get('emyx-j53e', limit=5000)
df_mill = pd.DataFrame.from_records(results_mill)

In [ ]:
# import grand lists
client_grand = Socrata('data.ct.gov', None)
results_grand = client_grand.get('8rr8-a322', limit=5000)
df_grand = pd.DataFrame.from_records(results_grand)

In [ ]:
# create keys to join the dataframes
df_mill['key'] = df_mill['municipality'] + df_mill['grand_list_year']
df_grand['key'] = df_grand['town_name'] + df_grand['grand_list_year'] 

In [ ]:
# join dataframes
df_join = df_mill.set_index('key').join(df_grand.set_index('key'), how='inner', lsuffix='_mill', rsuffix='_grand').sort_values(by=['key'])

In [ ]:
# inspect the new dataframe
df_join

In [ ]:
# inspect nulls
mill_rate_real_personal_and_mill_rate_are_null = df_join[df_join.mill_rate_real_personal.isnull() & df_join.mill_rate.isnull()]
print(len(mill_rate_real_personal_and_mill_rate_are_null))

In [ ]:
# drop nulls
df_drop = df_join.drop(mill_rate_real_personal_and_mill_rate_are_null.index)

In [ ]:
# drop Stamford outlier
df_drop = df_drop.drop('Stamford2019')

In [ ]:
# new column - mill_real
df_new_column = df_drop
df_new_column['mill_real'] = df_new_column['mill_rate_real_personal']

# if mill_rate_real_property is null, use mill_rate instead
df_new_column['mill_real'].fillna(df_new_column['mill_rate'], inplace = True)

In [ ]:
# convert columns to numeric
df = df_new_column.apply(pd.to_numeric, errors='ignore')

In [ ]:
# inspect data
print('Mill Rate - mean', '\n', df.groupby('fiscal_year')['mill_real'].mean(), '\n')
print('Grand List - mean', '\n', df.groupby('fiscal_year')['total_equalized'].mean(), '\n')

In [ ]:
# create new dataframe
df_pc = df.sort_values(by=['town_name', 'fiscal_year'])

In [ ]:
# new column
all_towns = pd.Series([])

for x in df_pc.town_name.unique():
    town_data = pd.Series(df_pc[df_pc.town_name == x].mill_real.pct_change())
    all_towns = pd.concat([all_towns, town_data])

df_pc['mill_real_pct_change'] = all_towns

In [ ]:
# new column
all_towns = pd.Series([])

for x in df_pc.town_name.unique():
    town_data = pd.Series(df_pc[df_pc.town_name == x].total_equalized.pct_change())
    all_towns = pd.concat([all_towns, town_data])

df_pc['total_equalized_pct_change'] = all_towns

## Town Data & Graphs

### SELECT A TOWN:

In [ ]:
# get user input
print(df_pc['town_name'].unique())
town = input('Enter town name:').title()

In [ ]:
# display means
print(town, '- Mill Rate - mean')
print(df[df.town_name == town].groupby('fiscal_year')['mill_real'].mean(), '\n')
print(town, '- Grand List - mean')
print(df[df.town_name == town].groupby('fiscal_year')['total_equalized'].mean(), '\n')

In [ ]:
# plot mill rate and grand list
y_mrate = df[df.town_name == town].groupby('fiscal_year')['mill_real'].mean()
y_glist = df[df.town_name == town].groupby('fiscal_year')['total_equalized'].mean()

fig, ax1 = plt.subplots()
fig.suptitle(town)

color = 'tab:red'
ax1.set_xlabel('Fiscal Year')
ax1.set_ylabel('Mill Rate', color=color)
ax1.plot(y_mrate, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Grand List', color=color)  # we already handled the x-label with ax1
ax2.plot(y_glist, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
# display percent change
print(town, '- Mill Rate - percent change')
print(df_pc[df_pc.town_name == town]['mill_real_pct_change'], '\n')
print(town, '- Grand List - percent change')
print(df_pc[df_pc.town_name == town]['total_equalized_pct_change'], '\n')

In [ ]:
# plot mill rate and grand list percent change
x = df_pc[df_pc.town_name == town]['total_equalized_pct_change']*100
y = df_pc[df_pc.town_name == town]['mill_real_pct_change']*100

plt.title(town + ' - Percent Change - mill rate') 
plt.xlabel('Grand List') 
plt.ylabel('Mill Rate') 
plt.scatter(x,y) 
plt.axhline(0,color='red') # x = 0
plt.axvline(0,color='red') # y = 0
plt.show()

In [ ]:
# scenario count - mill rate and grand list
q1 = df_pc[(df_pc.town_name == town) & (df_pc.total_equalized_pct_change < 0) & (df_pc.mill_real_pct_change > 0)]
q2 = df_pc[(df_pc.town_name == town) & (df_pc.total_equalized_pct_change > 0) & (df_pc.mill_real_pct_change > 0)]
q3 = df_pc[(df_pc.town_name == town) & (df_pc.total_equalized_pct_change > 0) & (df_pc.mill_real_pct_change < 0)]
q4 = df_pc[(df_pc.town_name == town) & (df_pc.total_equalized_pct_change < 0) & (df_pc.mill_real_pct_change < 0)]
pos_0 = df_pc[(df_pc.town_name == town) & (df_pc.total_equalized_pct_change > 0) & (df_pc.mill_real_pct_change == 0)]
neg_0 = df_pc[(df_pc.town_name == town) & (df_pc.total_equalized_pct_change < 0) & (df_pc.mill_real_pct_change == 0)]
zeros = df_pc[(df_pc.town_name == town) & (df_pc.total_equalized_pct_change == 0) & (df_pc.mill_real_pct_change == 0)]

print(town + " - scenarios - mill rate")
print('  grand - mill + ', len(q1), '\n', ' grand + mill + ', len(q2), '\n', '*grand + mill - ', len(q3), '\n', ' grand - mill - ', len(q4), '\n', ' grand + mill 0 ', len(pos_0), '\n', ' grand - mill 0 ', len(neg_0), '\n', ' grand 0 mill 0 ', len(zeros))

In [ ]:
# correlation - mill rate and grand list
df_cor_town = df_pc[df_pc.town_name == town]
df_cor_town = df_cor_town[['mill_real_pct_change', 'total_equalized_pct_change']]
df_matrix = df_cor_town.corr().round(2)

sns.heatmap(df_matrix, annot=True, vmax = 1, vmin = -1, center = 0, cmap = 'vlag')
plt.title(town + ' - correlations - mill rate')
plt.yticks(rotation='horizontal')
plt.show()

## State Data & Graphs

In [ ]:
# plot mill rate and grand list
y_mrate = df.groupby('fiscal_year')['mill_real'].mean()
y_glist = df.groupby('fiscal_year')['total_equalized'].mean()

fig, ax1 = plt.subplots()
fig.suptitle('All Towns - mill rate')

color = 'tab:red'
ax1.set_xlabel('Fiscal Year')
ax1.set_ylabel('Mill Rate', color=color)
ax1.plot(y_mrate, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('Grand List', color=color)  # we already handled the x-label with ax1
ax2.plot(y_glist, color=color)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()

In [ ]:
# display percent change
print('All Towns - Mill Rate - percent change')
print(df_pc['mill_real_pct_change'], '\n')
print('All Towns - Grand List - percent change')
print(df_pc['total_equalized_pct_change'], '\n')

In [ ]:
# plot mill rate and grand list percent change
x = df_pc['total_equalized_pct_change']*100
y = df_pc['mill_real_pct_change']*100

plt.title('All Towns - Percent Change - mill rate') 
plt.xlabel('Grand List') 
plt.ylabel('Mill Rate') 
plt.scatter(x,y) 
plt.axhline(0,color='red') # x = 0
plt.axvline(0,color='red') # y = 0
plt.show()

In [ ]:
# scenario count - mill rate and grand list
q1 = df_pc[(df_pc.total_equalized_pct_change < 0) & (df_pc.mill_real_pct_change > 0)]
q2 = df_pc[(df_pc.total_equalized_pct_change > 0) & (df_pc.mill_real_pct_change > 0)]
q3 = df_pc[(df_pc.total_equalized_pct_change > 0) & (df_pc.mill_real_pct_change < 0)]
q4 = df_pc[(df_pc.total_equalized_pct_change < 0) & (df_pc.mill_real_pct_change < 0)]
pos_0 = df_pc[(df_pc.total_equalized_pct_change > 0) & (df_pc.mill_real_pct_change == 0)]
neg_0 = df_pc[(df_pc.total_equalized_pct_change < 0) & (df_pc.mill_real_pct_change == 0)]
zeros = df_pc[(df_pc.total_equalized_pct_change == 0) & (df_pc.mill_real_pct_change == 0)]

print('All Towns - scenarios - mill rate')
print('  grand - mill + ', len(q1), '\n', ' grand + mill + ', len(q2), '\n', '*grand + mill - ', len(q3), '\n', ' grand - mill - ', len(q4), '\n', ' grand + mill 0 ', len(pos_0), '\n', ' grand - mill 0 ', len(neg_0), '\n', ' grand 0 mill 0 ', len(zeros))

In [ ]:
# correlation - mill rate and grand list
df_cor_state = df_pc[['mill_real_pct_change', 'total_equalized_pct_change']]
df_matrix = df_cor_state.corr().round(2)

sns.heatmap(df_matrix, annot=True, vmax = 1, vmin = -1, center = 0, cmap = 'vlag')
plt.title('All Towns - correlations - mill rate')
plt.yticks(rotation='horizontal')
plt.show()